In [1]:
import os
import sys
import controlflow as cf

from agent_utils import *
from agent_tools import *

from extensive_search import run_research
from rich import print as pprint
FLASH2_MODEL = "google/" + config.FLASH2_MODEL



In [2]:
query = f"""
We need an detailed and extensive overview of existing systems for running pretrained model (e.g., transformer models) up to 100 million parameters (inference only).
To accomplish this, we plan to do a deep research by processing many webpages, PDFs, acticles etc. using LLM APIs (e.g., Google Gemini, GPT4o etc.).

We use an existing Python agent framework (e.g., PydanticAI, LangChain, Controlflow) to build the system.

We have access to different types of search tools:
- Google Search API (general search with time range search)
- Google Schoolar API (search papers)
- Google News API (search news articles)
- Could be expanded in the future.

For this task, we plan to build our own deep research agent system that uses LLMs to process a workflow.

The workflow has these basic working steps or tasks:
- Based on a search query (user input) generate a research plan/instruction and suitable search queries for the search tools.
- Use search APIs for generated search queries to collect information.
- Use a reasoning model (LLM + thinking) to select suitable URLs to extract information from.
- Use web crawlers to comvert HTML or PDF content to markdown text for the LLMs.
- For the first markdown document use an LLM to rate the relevance and quality of the document.
- For the first document also extract a summary of all relevant and important information.
- Save the document with rating and contectualized summary.
- For the second markdown document use the contectualized summary from the previous document, to genrate a rating and a new contextualized summary.
- Repeat the earlier steps for each downloaded document.
- Stop when a cerain document budget is reached.
- Use the original user query and the compressed/accumulated summary to create a report based on the generated research plan/instruction.

What do you think about this approach? How would you improve it?

"""

In [3]:
reasoningModel = ReasoningModel()
basicSearchModel = BasicSearchModel()


user_query = "We need a detailed and extensive overview of existing systems (e.g., ARM, FPGA, NPU, GPUs, etc.) for running pretrained model (e.g., transformer models) up to 100 million parameters (inference only)."

query = f"""
{user_query}

We can use different types of search tools:
- Google Search API (general search with time range search)
- Google Schoolar API (search papers)
- Google News API (search news articles)

Come up with suitable search queries for these tools.

The format should following this structure:

Google Search:
- query 1
- query 2
- etc.

Google Schoolar API:
- etc.


"""

if not (response := load_data("embedded_devices_search")):
    response = reasoningModel(query)
    save_data(response, "embedded_devices_search")
    cprint(response)

In [4]:

class SearchQueries(BaseModel):
    google_search: Optional[list[str]] = Field(description="The search queries from Google Search.")
    scholar_search: Optional[list[str]] = Field(description="The search queries from Google Scholar.")
    news_search: Optional[list[str]] = Field(description="The search queries from Google News.")

query_extraction = BasicAgent(result_type=SearchQueries, system_prompt="Extract the search queries from a given text (only the queries, nothing else).")


result = await query_extraction(f"{response}")

google_search_queries = result.data.google_search
scholar_search_queries = result.data.scholar_search
news_search_queries = result.data.news_search

/home/dplaia/Projekte/DeepResearch/agent_tools.py:39: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  return await self.agent.run(user_input)


## Google Search

In [6]:
url_list = []

In [9]:

if not (results := load_data("google_search_results")):
    results = []

    for google_search_query in google_search_queries:
        search_results = await google_general_search_async(google_search_query.replace("\"", ""), time_span = TimeSpan.YEAR)

        results.append(search_results)

    for result in results:
        organic = result['organic']

        urls = [t['link'] for t in organic]

        for url in urls:
            if not url in url_list:
                url_list.append(url)

    save_data(results, "google_search_results")

## Scholar Search

In [15]:
if not (results := load_data("scholar_search_results")):
    results = []

    for scholar_query in scholar_search_queries:
        search_results = await google_general_search_async(scholar_query.replace("\"", ""))

        results.append(search_results)

    for result in results:
        organic = result['organic']

        urls = [t['link'] for t in organic]

        for url in urls:
            if not url in url_list:
                url_list.append(url)
    save_data(results, "scholar_search_results")

## Scrape content

In [ ]:
if not (markdown_pages := load_data("markdown_pages")):

    markdown_pages = {}

    for url in url_list:
        print(url)

        try:
            markdown_text = await crawl4ai_website_async(url)
            markdown_pages[url] = markdown_text
        except Exception as e:
            print(e)
            continue

    save_data(markdown_pages, "markdown_pages")

In [29]:
memory = 0
for page in markdown_pages.values():
    memory += sys.getsizeof(page)

print(f"Memory needed: {memory/1024/1024:.2f} MB")

Memory needed: 24.91 MB


## Process content